# Exploration

### Gold layer
- dim_customer: customer_id, 
  - crm_customer: customer_id, customer_number, FN, LN, MS, gender, created_date
  - erp_customer_location: customer_id, country
  - erp_customers: customer_id, birth_date, gender

In [0]:
query = """
SELECT 
ROW_NUMBER() OVER (ORDER BY crm.customer_id) AS customer_key,
crm.customer_id,
crm.customer_number,
crm.first_name,
crm.last_name,
crm.marital_status,
CASE 
  WHEN crm.gender <> 'n/a' THEN crm.gender
  ELSE COALESCE(erp.gender, 'n/a')
  END AS gender,
erp.birth_date as birthdate,
crm.created_date as create_date
FROM workspace.silver.crm_customers crm
JOIN workspace.silver.erp_customers erp 
ON crm.customer_number = erp.customer_id
JOIN workspace.silver.erp_customer_location loc 
ON crm.customer_number = loc.customer_id
"""

df = spark.sql(query)

In [0]:
df.limit(10).display()

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_customers")